# Лабораторна Робота 1
### З дисципліни "Методи глибинного навчання"
##### Виконала Неділько Дарина, група КМ-91
##### Варіант 9
**Тема:** «Нейромережеве розпізнавання кібератак»
**Мета:** Розробка програмного забезпечення для реалізації нейронних мереж типу PNN та ДШП, призначених для розпізнавання кібератак, сигнатури яких представлені в базах даних KDD-99 або NLS KDD

**Завдання:** Розпізнавання мережевої кібератаки типу imap за допомогою ДШП


Для тренування нейронної мережі відбуватиметься на основі датасету KDD-99 або ж [KDD Cup 1999 Data](https://www.kaggle.com/datasets/galaxyh/kdd-cup-1999-data). Це датасет, у якому зберігається інформація про мережеві вторгнення. За умовою завдання ми будемо розглядати тільки записи з типом атаки **normal та imap**. Для початку завантажимо потрібні дані.

Тренування відбувалося на пристрої з такими характеристиками: 
 - процесор: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz   1.99 GHz
 - RAM : 16 GB / 15.9 usable
 - system type: 64-bit OS, x64-based processor
 - OS : Windows 11 Pro

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

# назви колонок датасету
columns=['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot','num_failed_logins','logged_in','num_compromised','root_shell',
'su_attempted','num_root','num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','attack_type']

# дістаємо дані
path = "archive/kddcup.data_10_percent.gz"
data_uncleaned = pd.read_csv(path ,names=columns)
print(f'Розмір датасету: параметри {data_uncleaned.shape[1]} та кількість записів {data_uncleaned.shape[0]}')


Розмір датасету: параметри 42 та кількість записів 494021


Отримаємо потрібні типи атак imap. and normal. Дані характеризуються 41 вхідним та 1 вихідним параметром. Вихідним параметром виступа attack_type. Шматок даних нижче(з типом imap):

In [2]:
print(f'Данi для типу imap.: \n')
data_uncleaned[data_uncleaned.attack_type == 'imap.']

Данi для типу imap.: 



,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type
40723,31,tcp,imap4,SF,1345,10036,0,0,0,0,...,1,0.0,0.01,0.00,0.0,0.00,0.00,0.03,0.0,imap.
76313,0,tcp,imap4,SH,0,0,0,0,0,0,...,1,1.0,0.00,1.00,0.0,1.00,1.00,0.00,0.0,imap.
76314,0,tcp,imap4,SH,0,0,0,0,0,0,...,2,1.0,0.00,1.00,0.0,1.00,1.00,0.00,0.0,imap.
76315,0,tcp,imap4,SF,0,0,0,0,0,0,...,3,1.0,0.00,1.00,0.0,0.67,0.67,0.00,0.0,imap.
76316,0,tcp,imap4,SF,0,0,0,0,0,2,...,4,1.0,0.00,1.00,0.0,0.50,0.50,0.00,0.0,imap.
76317,41,tcp,imap4,SF,1334,162,0,0,0,0,...,5,1.0,0.00,0.20,0.0,0.40,0.40,0.00,0.0,imap.
76318,0,tcp,imap4,S0,0,0,0,0,0,0,...,6,1.0,0.00,0.17,0.0,0.50,0.50,0.00,0.0,imap.
76319,0,tcp,imap4,SH,0,0,0,0,0,0,...,7,1.0,0.00,0.71,0.0,0.57,0.57,0.00,0.0,imap.
76320,0,tcp,imap4,SH,0,0,0,0,0,0,...,8,1.0,0.00,0.75,0.0,0.62,0.62,0.00,0.0,imap.
76321,0,tcp,imap4,SF,0,0,0,0,0,0,...,9,1.0,0.00,0.78,0.0,0.56,0.56,0.00,0.0,imap.


Оскільки у нас всього 12 прикладів цієї атаки, тоб збалансуємо вибірку, зменшивши представників атаки normal. Наприклад 1:3, тобто для 'normal.' буде 36 записів, а для 'imap.' 12.

In [3]:
d1_normal = data_uncleaned[data_uncleaned.attack_type == 'normal.'].head(36)
d2_imap = data_uncleaned[data_uncleaned.attack_type == 'imap.']

selected_data = pd.concat([d1_normal,d2_imap]).sample(frac = 1)
print(f'Отриманий у ході перетворень датасет: \n')
selected_data.head(10)

Отриманий у ході перетворень датасет: 



,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type
76316,0,tcp,imap4,SF,0,0,0,0,0,2,...,4,1.0,0.0,1.00,0.00,0.50,0.50,0.0,0.0,imap.
21,0,tcp,http,SF,234,255,0,0,0,0,...,219,1.0,0.0,0.01,0.03,0.00,0.00,0.0,0.0,normal.
20,0,tcp,http,SF,256,1273,0,0,0,0,...,209,1.0,0.0,0.01,0.03,0.00,0.00,0.0,0.0,normal.
14,0,tcp,http,SF,241,259,0,0,0,0,...,149,1.0,0.0,0.07,0.04,0.00,0.00,0.0,0.0,normal.
76317,41,tcp,imap4,SF,1334,162,0,0,0,0,...,5,1.0,0.0,0.20,0.00,0.40,0.40,0.0,0.0,imap.
76320,0,tcp,imap4,SH,0,0,0,0,0,0,...,8,1.0,0.0,0.75,0.00,0.62,0.62,0.0,0.0,imap.
8,0,tcp,http,SF,210,151,0,0,0,0,...,89,1.0,0.0,0.12,0.04,0.00,0.00,0.0,0.0,normal.
76323,0,tcp,imap4,S1,1492,649186,0,0,0,0,...,11,1.0,0.0,0.09,0.00,0.55,0.55,0.0,0.0,imap.
27,0,tcp,http,SF,193,3991,0,0,0,0,...,255,1.0,0.0,1.00,0.05,0.00,0.00,0.0,0.0,normal.
76314,0,tcp,imap4,SH,0,0,0,0,0,0,...,2,1.0,0.0,1.00,0.00,1.00,1.00,0.0,0.0,imap.


Для початку треба розібратися з тим, що означають вхідні параметри:
- Duration: тривалість з'єднання ;
- protocol_type: тип протоколу з'єднання;
- service: служба та порт призначення;
- src_bytes: кількість байтів від джерела до призначення;
- dst_bytes: кількість байтів від призначення до джерела;
- flag: стату підключення(помилка\успішне);
- land: 1 якщо з'єднання з\до того ж хоста\порта;
- wrong_fragment: кількість "поганих" фрагментів;
- urgent: кількість термінових пакунків;
- hot: кількість "гарячих" індикаторів;
- num_failed_logins: кількість провалених спроб залогінитися;
- logged_in: 1 успішний логін, 0 неуспішний;
- num_compromised: кількість скомпроментованих пунктів;
- num_compromised: кількість скомпроментованих пунктів;
- root_shell: 1 якщо отримана коренева оболонка / 0 в іншому випадку;
- su_attempted: 1 при спробі команди «su root»/ 0 в іншому випадк;
- num_root: кількість «root» доступів;
- num_file_creations: кількість операцій створення файлів;
- num_shells: кількість підказок оболонки;
- num_access_files: кількість операцій з файлами контролю доступу;
- num_outbound_cmds: кількість вихідних команд у сеансі ftp;
- is_hot_login: 1, якщо логін належить до «hot» списку, 0 в іншому випадку;
- is_guest_login: 1, якщо логін є «guest» логіном/ 0 в іншому випадку;
- 
.
.
.




Тепер закодуємо параметри, що представлені у текстовому форматі. Для початку виведемо їх:

In [4]:
to_code = list(selected_data.select_dtypes(include=['object']).columns)
to_code

['protocol_type', 'service', 'flag', 'attack_type']

Тепер створюємо словник з кодуваннями для кожного текстового поля.

In [5]:

coders_nested = dict()
for point in to_code:
    coders_nested[point]={}
    for i,name in enumerate(selected_data[point].unique()):
        coders_nested[point][name] = i

for p in coders_nested: print(f'{p} :\n\t\t{coders_nested[p]}\n')

protocol_type :
		{'tcp': 0}

service :
		{'imap4': 0, 'http': 1}

flag :
		{'SF': 0, 'SH': 1, 'S1': 2, 'S0': 3}

attack_type :
		{'imap.': 0, 'normal.': 1}



Тепер кодуємо сам датасет

In [6]:
coded_df = selected_data.reset_index(drop = True)

for p in coders_nested:
    print(p)
#     coded_df[p]=coded_df[p].astype(str)
    coded_df[p] = coded_df[p].replace(coders_nested[p])
coded_df.head(10)


protocol_type
service
flag
attack_type


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type
0,0,0,0,0,0,0,0,0,0,2,...,4,1.0,0.0,1.00,0.00,0.50,0.50,0.0,0.0,0
1,0,0,1,0,234,255,0,0,0,0,...,219,1.0,0.0,0.01,0.03,0.00,0.00,0.0,0.0,1
2,0,0,1,0,256,1273,0,0,0,0,...,209,1.0,0.0,0.01,0.03,0.00,0.00,0.0,0.0,1
3,0,0,1,0,241,259,0,0,0,0,...,149,1.0,0.0,0.07,0.04,0.00,0.00,0.0,0.0,1
4,41,0,0,0,1334,162,0,0,0,0,...,5,1.0,0.0,0.20,0.00,0.40,0.40,0.0,0.0,0
5,0,0,0,1,0,0,0,0,0,0,...,8,1.0,0.0,0.75,0.00,0.62,0.62,0.0,0.0,0
6,0,0,1,0,210,151,0,0,0,0,...,89,1.0,0.0,0.12,0.04,0.00,0.00,0.0,0.0,1
7,0,0,0,2,1492,649186,0,0,0,0,...,11,1.0,0.0,0.09,0.00,0.55,0.55,0.0,0.0,0
8,0,0,1,0,193,3991,0,0,0,0,...,255,1.0,0.0,1.00,0.05,0.00,0.00,0.0,0.0,1
9,0,0,0,1,0,0,0,0,0,0,...,2,1.0,0.0,1.00,0.00,1.00,1.00,0.0,0.0,0


Тепер проведемо нормалізацію параметрів за формулою: $$X*=\frac{X-min(X)}{max(X)-min(X)}$$

In [7]:
for param in coded_df:
    mins = coded_df[param].max()
    maxs = coded_df[param].min()
#     print(f'\nParameter {param} \n -> min {mins} and max {maxs}')
    coded_df[param]= coded_df[param].map(lambda x: (x-mins)/(maxs-mins))

coded_df.head(10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type
0,1.0,NaN,1.0,1.000000,1.000000,1.000000,NaN,NaN,NaN,-0.0,...,0.988189,-0.0,1.0,-0.00,1.0,0.50,0.50,1.0,NaN,1.0
1,1.0,NaN,-0.0,1.000000,0.843164,0.999607,NaN,NaN,NaN,1.0,...,0.141732,-0.0,1.0,0.99,0.4,1.00,1.00,1.0,NaN,-0.0
2,1.0,NaN,-0.0,1.000000,0.828418,0.998039,NaN,NaN,NaN,1.0,...,0.181102,-0.0,1.0,0.99,0.4,1.00,1.00,1.0,NaN,-0.0
3,1.0,NaN,-0.0,1.000000,0.838472,0.999601,NaN,NaN,NaN,1.0,...,0.417323,-0.0,1.0,0.93,0.2,1.00,1.00,1.0,NaN,-0.0
4,-0.0,NaN,1.0,1.000000,0.105898,0.999750,NaN,NaN,NaN,1.0,...,0.984252,-0.0,1.0,0.80,1.0,0.60,0.60,1.0,NaN,1.0
5,1.0,NaN,1.0,0.666667,1.000000,1.000000,NaN,NaN,NaN,1.0,...,0.972441,-0.0,1.0,0.25,1.0,0.38,0.38,1.0,NaN,1.0
6,1.0,NaN,-0.0,1.000000,0.859249,0.999767,NaN,NaN,NaN,1.0,...,0.653543,-0.0,1.0,0.88,0.2,1.00,1.00,1.0,NaN,-0.0
7,1.0,NaN,1.0,0.333333,-0.000000,-0.000000,NaN,NaN,NaN,1.0,...,0.960630,-0.0,1.0,0.91,1.0,0.45,0.45,1.0,NaN,1.0
8,1.0,NaN,-0.0,1.000000,0.870643,0.993852,NaN,NaN,NaN,1.0,...,-0.000000,-0.0,1.0,-0.00,-0.0,1.00,1.00,1.0,NaN,-0.0
9,1.0,NaN,1.0,0.666667,1.000000,1.000000,NaN,NaN,NaN,1.0,...,0.996063,-0.0,1.0,-0.00,1.0,-0.00,-0.00,1.0,NaN,1.0


У нас є колонки з порожніми значеннями, для тренування моделі вони не матимуть користі, тому було прийнято рішення їх видалити. А також розділимо датасет на feachures та ladels.

In [8]:
df_xy = coded_df.dropna(axis=1)
df_y = df_xy.attack_type
df_x = df_xy.drop(columns=['attack_type'])

df_xy.head(5)

,duration,service,flag,src_bytes,dst_bytes,hot,logged_in,num_compromised,num_root,count,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,attack_type
0,1.0,1.0,1.0,1.000000,1.000000,-0.0,1.0,1.0,1.0,0.863636,...,0.988189,0.988189,-0.0,1.0,-0.00,1.0,0.5,0.5,1.0,1.0
1,1.0,-0.0,1.0,0.843164,0.999607,1.0,-0.0,1.0,1.0,0.818182,...,0.673228,0.141732,-0.0,1.0,0.99,0.4,1.0,1.0,1.0,-0.0
2,1.0,-0.0,1.0,0.828418,0.998039,1.0,-0.0,1.0,1.0,0.272727,...,0.712598,0.181102,-0.0,1.0,0.99,0.4,1.0,1.0,1.0,-0.0
3,1.0,-0.0,1.0,0.838472,0.999601,1.0,-0.0,1.0,1.0,1.000000,...,0.948819,0.417323,-0.0,1.0,0.93,0.2,1.0,1.0,1.0,-0.0
4,-0.0,1.0,1.0,0.105898,0.999750,1.0,1.0,1.0,1.0,1.000000,...,0.984252,0.984252,-0.0,1.0,0.80,1.0,0.6,0.6,1.0,1.0


In [9]:
# test cell - must be deleted 

df_x.head(5)

,duration,service,flag,src_bytes,dst_bytes,hot,logged_in,num_compromised,num_root,count,...,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate
0,1.0,1.0,1.0,1.000000,1.000000,-0.0,1.0,1.0,1.0,0.863636,...,0.72,0.988189,0.988189,-0.0,1.0,-0.00,1.0,0.5,0.5,1.0
1,1.0,-0.0,1.0,0.843164,0.999607,1.0,-0.0,1.0,1.0,0.818182,...,1.00,0.673228,0.141732,-0.0,1.0,0.99,0.4,1.0,1.0,1.0
2,1.0,-0.0,1.0,0.828418,0.998039,1.0,-0.0,1.0,1.0,0.272727,...,1.00,0.712598,0.181102,-0.0,1.0,0.99,0.4,1.0,1.0,1.0
3,1.0,-0.0,1.0,0.838472,0.999601,1.0,-0.0,1.0,1.0,1.000000,...,1.00,0.948819,0.417323,-0.0,1.0,0.93,0.2,1.0,1.0,1.0
4,-0.0,1.0,1.0,0.105898,0.999750,1.0,1.0,1.0,1.0,1.000000,...,1.00,0.984252,0.984252,-0.0,1.0,0.80,1.0,0.6,0.6,1.0


Для подальної роботи необхідно розділити вибірку на навчальну та тренувальну у співвідношення 9:1. Для цього скористаємося методом train_test_split з модулю sklearn.model_selection.

In [10]:
x_train,x_test,y_train,y_test=train_test_split(df_x,df_y,test_size=0.2,random_state=123)

In [11]:
print(f'Розміри датасетів:\n X Train\n{x_train.shape}\n\n\n Y Train\n{y_train.shape}\n\n\nX test\n{x_test.shape}\n\n\nY Test\n{y_test.shape}')

Розміри датасетів:
 X Train
(38, 24)


 Y Train
(38,)


X test
(10, 24)


Y Test
(10,)


Необхідно визначити будову нейронної мережі. У вхідному шарі буде стільки нейронів, скільки параметрів ми даємо на вхід. У вихідному шарі буде 1 нейрон. Для обчислення кількості нейронів у прихованому шарі необхідно скористатися формулами для визначення меж інтервалу ефективної кількості схованих нейронів.
$$N_h^A = round((0.4 N_x + 0.2)\frac{\sqrt{PN_x}}{N_x + N_y})$$ $$N_h^B = round(\frac{2\sqrt{PN_x}}{N_y}),$$
 - $N_h^A,N_h^B$ - межі інтервалу;
 - $N_x$ - кількість вхідних нейронів;
 - $N_y$ - кількість вхідних нейронів;
 - $P$ - кількість навчальних прикладів.


In [12]:
import math
# активаційна функція сигноїд
sign = lambda x: 1/(1+math.exp(-x))

# кількість вхідних нейронів, вихідних 1
P, N_x = x_train.shape
N_y = 1
# кількість нейронів у прихованому шарі
N_hA = (0.4*N_x+0.2) * ((math.sqrt(P*N_x))/(N_y+N_x))
N_hB = (2*math.sqrt(P*N_x)/N_y)



print(f'Kількість вхідних нейронів {N_x}, межі інтервалу кількості нейронів у прихованому шарі ({round(N_hA,0)},{round(N_hB,0)}).\nВиходячи з оптимального діапазону було обрано реалізовувати {round(N_hA,0)+1} у прихованому шарі')

hid = int(round(N_hA,0)+1)


Kількість вхідних нейронів 24, межі інтервалу кількості нейронів у прихованому шарі (12.0,60.0).
Виходячи з оптимального діапазону було обрано реалізовувати 13.0 у прихованому шарі


In [27]:

# ДШП
def train(x, y, w1, w2,hid,st):
    temp = []
#     будуємо прихований шар - множимо вхідні 
# дані на відповідні вагові коефіціенти
    hidden_layer = np.dot(x, w1)
#     дізнаємося значення активаційної ф-ї для прихованого та вихідного шарів
    y_s = [sign(x_s) for x_s in hidden_layer]
    out = np.dot(y_s, w2)
    y_s.append(sign(out))
#     вираховуємо абсолютну похибку ітерації
    error = abs((y - y_s[-1]) / y)
    q13 = y_s[-1] * (1 - y_s[-1]) * (y - y_s[-1])
    # градієнтний спуск для кожного важка - приріст функції
    for k in range(hid):
        b = q13 * y_s[k]
        temp.append(w2[k] + b)
    for i in range(hid):
        q = y_s[i] * (1 - y_s[i]) * q13 * w2[i]
        for k in range(st):
            b = q * x[k]
            w1[k, i] += b
    w2 = []
    w2.extend(temp)
    return w1, w2, y_s[-1], error


# функція реалізації онлайн тренування з епохами(ітераціями)
'''
запускаємо цикл тренування ДШП(максимальна кількість ітерацій 500)
Для кожної ітерації виводиться помилка та номер ітерації(епохи)
'''
def online(x, y, w1, w2, hid, st):
    _y = []
    train_err = []
    epoch = 1 
    while epoch < 501:
        tempres, temperr = [], []
        for k in x.index:
            w1, w2, result, error = train(x.loc[k, :], y[k], w1, w2, hid, st)
            tempres.append(result)
            temperr.append(error)
        if epoch in [1, 100, 200,400, 500]:
            print("Епоха: ", epoch)
            print("Похибка: ", error)
        _y.append(tempres)
        train_err.append(temperr)
        epoch += 1
    return w1, w2

# ініціалізація вагів
w_1 = np.random.rand(N_x, hid)

w_2 = np.random.rand(hid)

# результат тренування ДШП
w = online(x_train.iloc[:], y_train.iloc[:], w_1, w_2, hid, N_x)


Епоха:  1
Похибка:  inf
Епоха:  100
Похибка:  inf
Епоха:  200
Похибка:  inf
Епоха:  400
Похибка:  inf
Епоха:  500
Похибка:  inf


In [ ]:
# натренована модель для розпізнавання
def model_(x, w1, w2):
    y_pred = []
    for i in range(len(x)):
        hidden_layer = np.dot(x.iloc[i:i+1, :], w1)
        y_s = [sign(x_s) for x_s in hidden_layer[0]]
        out = np.dot(y_s, w2)
        res = sign(out)
        y_s.append(res)
        y_pred.append(res)
    return y_pred

# передбачені у для тренувальних та тестових даних
y_pred_test = model_(x_test.iloc[:], w[0], w[1])
y_pred_train = model_(x_train.iloc[:], w[0], w[1])

y_pred_test = [abs(round(x)) for x in y_pred_test]
y_pred_train = [abs(round(x)) for x in y_pred_train]

test_guess = [abs(list(y_test)[i])==y_pred_test[i] for i in range(len(y_test))]
train_guess = [abs(list(y_train)[i])==y_pred_train[i] for i in range(len(y_train))]

res1 = {'Yr': y_train, 
        'Y predicted': y_pred_train,
       'Рівні': train_guess}
    
result1 = pd.DataFrame(res1)
res2 = {'Yr': y_test, 
    'Y predicted': y_pred_test,
       'Рівні': test_guess}

result2 = pd.DataFrame(res2)

print(f'Для тестових даних: \n{result2}\n\n Похибка передбачення {len(result2[result2["Рівні"]==False])/len(result2)}\n\nДля тренувальних даних: \n{result1}\n\n Похибка передбачення {len(result2[result2["Рівні"]==False])/len(result2)}')

### Висновки:
У ході лабораторної роботи було відпрацьовано здобуті у ході 1 та 2  лабораторних робіть навички. Для роботи ДШП попередньо було почищено, збалансовано та нормалізовані дані. Точність розпізнавання побудованої моделі дуже висока. Похибку було пораховано за формулою кількість неправильно розпізнаних/кількість тестових прикладів і вона сягала 0. Як для тестових, так і для тренувальниї даних. Розподіл елементів по вибірках відбувався за допомогою рандомайзеру.